In [1]:
from urllib.request import *
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import scipy.stats as stats
import pylab as pl
import requests
from geopy.geocoders import Nominatim
import json
import folium
import os
import time
from tempfile import TemporaryFile
from datetime import datetime
import matplotlib.pyplot as plt
from collections import Counter
from scipy import sparse, stats, spatial
import scipy.sparse.linalg
import networkx as nx
from pylab import rcParams
%matplotlib inline

---
# Abstract

Question: What is the recipy a player should follow to increase his market value the most?

Data: from [TransferMarket](https://www.transfermarkt.co.uk/), we have scrapped the clubs that have a total market value above 200 million euros. From these clubs, we downloaded the html files of their football players. This include the nationality, birthdate, transfer history, transfer fee etc.

Purpose: We want to identify what are the most important factors that might increase the value of a player when transfering from one club to another.


## Loading clubs data

In [114]:
with open("../scraper/data/clubs.json", "r") as in_file:
    clubs = json.load(in_file)
print("There are " + str(len(clubs)) + " clubs")
clubs[0]

There are 418 clubs


{'href': '/chelsea-fc/kader/verein/631/saison_id/2017',
 'market_value': '631,90 Mill. €',
 'name': 'Chelsea FC',
 'players': [{'href': '/thibaut-courtois/profil/spieler/108390',
   'id': '108390',
   'name': 'Thibaut Courtois'},
  {'href': '/willy-caballero/profil/spieler/19948',
   'id': '19948',
   'name': 'Willy Caballero'},
  {'href': '/eduardo/profil/spieler/34159', 'id': '34159', 'name': 'Eduardo'},
  {'href': '/matej-delac/profil/spieler/60220',
   'id': '60220',
   'name': 'Matej Delac'},
  {'href': '/david-luiz/profil/spieler/46741',
   'id': '46741',
   'name': 'David Luiz'},
  {'href': '/antonio-rudiger/profil/spieler/86202',
   'id': '86202',
   'name': 'Antonio Rüdiger'},
  {'href': '/andreas-christensen/profil/spieler/196948',
   'id': '196948',
   'name': 'Andreas Christensen'},
  {'href': '/gary-cahill/profil/spieler/27511',
   'id': '27511',
   'name': 'Gary Cahill'},
  {'href': '/marcos-alonso/profil/spieler/112515',
   'id': '112515',
   'name': 'Marcos Alonso'},
  

## Loading players

In [116]:
with open("../scraper/data/players_ref.json", "r") as in_file:
    players_ref_list = json.load(in_file)

club_ids = []
for club in clubs:
    club_ids.append(int(club['href'].split("/")[-3]))
    
print ("We have {} players".format(len(players_ref_list)))

We have 12075 players


In [87]:
# This function will transform the fees of a player from a string to a number so that it can be used as a weight for the edges
def fix_fee(player_fee):
    try:
        player_fee = player_fee.replace(":",": ").replace(",",".")
    except:
        pass
    if re.search(r'\d+.\d+', player_fee) is not None:
        fee = float(re.search(r'\d+.\d+', player_fee).group())
    else:
        fee = 0
    
    if fee > 0:
        if "Mill." in player_fee:
            processed_fee = int(fee*1000000)
        elif "Th." in player_fee:
            processed_fee = int(fee*1000)
        else:
            processed_fee = int(fee)
    else:
        processed_fee = 0
    
    return processed_fee

# Returns the year of the transfer
def get_year(date):
    return date[-4:]

# Define method to parse player given player url
def parsePlayer(player_ref):    
    playerID = player_ref.split("/")[-1]
    
    with open("../scraper/data/players/" + playerID + "/page.html") as in_file:
        player_page = json.load(in_file)
    
    response = BeautifulSoup(player_page, 'html.parser')
    
    playerInfos = str(response.find("table", {"class":"auflistung"}))
    player = {}
    player["href"] = player_ref
    try:
        player["number"] = response.find("span", {"class":"dataRN"}).text
    except:
        player["number"] = None
    player["name"] = response.find("h1", {"itemprop":"name"}).text
    player["player_id"] = player_ref.split("/")[-1]
    position = BeautifulSoup(playerInfos.split("Position")[1], 'html.parser').find("td").text
    reg = re.compile( "[a-zA-Z -]")
    player["position"] = "".join(reg.findall(position))
    try:
        player["birthdate"] = BeautifulSoup(playerInfos.split("Date of birth")[1], 'html.parser').find("td").text
    except:
        player["birthdate"] = None
    player["nationality"] = BeautifulSoup(playerInfos.split("Nationality")[1], 'html.parser').find("td").find("img")["title"]
    player["current_club"] = BeautifulSoup(playerInfos.split("Current club")[1], 'html.parser').find("td").find_all("a")[-1].text

    try:
        transfers = []
        trans = response.find("div",{"class" : "box transferhistorie"}).find("table").find("tbody").find_all("tr", {"class":"zeile-transfer"})

        for t in trans:
            transfer = {}
            transfer["player"] = player_ref.split("/")[-1]
            transfer["date"] = t.find_all("td", {"class":"zentriert hide-for-small"})[1].text
            transfer["from"] = t.find_all("td", {"class":"no-border-rechts vereinswappen"})[0].find("a")["id"]
            transfer["to"] = t.find_all("td", {"class":"no-border-rechts vereinswappen"})[1].find("a")["id"]
            if (t.find("td", {"class":"zelle-abloese"}).text) == "End of loan" or t.find("td", {"class":"zelle-abloese"}).text =="Loan":
                transfer["fee"] = t.find("td", {"class":"zelle-mw"}).text
            else:
                transfer["fee"] = t.find("td",{"class":"zelle-abloese"}).text
            transfer["fee"] = fix_fee(transfer["fee"])
            transfers.append(transfer)
    except:
        transfers = None
        
    return player, transfers


def get_club_value(club_id):
    for club in clubs:
        if (int(club['href'].split("/")[-3]) == club_id):
            return fix_fee(club['market_value'])
    

In [6]:
# Example test of what the function would return
player_test = parsePlayer(players_ref_list[0])
player_test

({'birthdate': 'May 11, 1992 ',
  'current_club': 'Chelsea FC',
  'href': '/thibaut-courtois/profil/spieler/108390',
  'name': 'Thibaut Courtois',
  'nationality': 'Belgium',
  'number': '#13',
  'player_id': '108390',
  'position': 'Goalkeeper'},
 [{'date': 'Jun 30, 2014',
   'fee': 25000000,
   'from': '13',
   'player': '108390',
   'to': '631'},
  {'date': 'Jul 27, 2011',
   'fee': 1200000,
   'from': '631',
   'player': '108390',
   'to': '13'},
  {'date': 'Jul 26, 2011',
   'fee': 8950000,
   'from': '1184',
   'player': '108390',
   'to': '631'},
  {'date': 'Jul 1, 2009',
   'fee': 0,
   'from': '3494',
   'player': '108390',
   'to': '1184'},
  {'date': 'Jul 1, 2008',
   'fee': 0,
   'from': '49393',
   'player': '108390',
   'to': '3494'},
  {'date': 'Jul 1, 2007',
   'fee': 0,
   'from': '34282',
   'player': '108390',
   'to': '49393'}])

## Build graph and dataframe of transfers

In [111]:
transfer = {'from_club_id': [],
            'to_club_id': [],
            'player_nationality': [],
            'value_increase': [],
            'player_stay_in_years': [],
            'club_market_value_from': [],
            'club_market_value_to': [],
            'transfer_year': [],
            'position': [],
            'birth_date_year': []
            }

G = nx.DiGraph()
# Go through all the players
for player_ref in players_ref_list:
    player = parsePlayer(player_ref)
    # Check that player has more than 1 transfer recorded
    if player[1] is not None and len(player[1])>1:
        # Get initial recorded transfer
        year_start = int(get_year(player[1][-1]["date"].strip()))
        # Loop over player transfers from oldest to most recent
        for index in range(2,len(player[1])+1):
            # Compute the number of years a player stayed in the club before he transfered
            player_stay = int(get_year(player[1][-index]["date"].strip())) - year_start
            # Update year_start
            year_start = int(get_year(player[1][-index]["date"].strip()))
            # Only consider the switch when a player stayed less than 5 years in a club
            if (player_stay <= 5):
                from_club = int(player[1][-index]["from"])
                to_club = int(player[1][-index]["to"])
                # Only considering transfers between clubs whose value is above 200 million euros
                if(from_club in club_ids and to_club in club_ids):
                    # Value increase/decrease in a player compared to previous transfer
                    value_increase = player[1][-index]["fee"] - player[1][-index+1]["fee"]
                    # Add entry to dataframe
                    transfer['from_club_id'].append(from_club)
                    transfer['to_club_id'].append(to_club)
                    transfer['player_nationality'].append(player[0]['nationality'])
                    transfer['value_increase'].append(value_increase)
                    transfer['player_stay_in_years'].append(player_stay)
                    transfer['club_market_value_from'].append(get_club_value(from_club))
                    transfer['club_market_value_to'].append(get_club_value(to_club))
                    transfer['transfer_year'].append(get_year(player[1][-index]['date'].strip()))
                    transfer['position'].append(player[0]['position'])
                    transfer['birth_date_year'].append(get_year(player[0]['birthdate'].strip()))
                    # Build graph from the transfer. Node is a club
                    # add two attributes: transfer fee between clubs and number of transfers
                    if G.has_edge(from_club, to_club):
                        G[from_club][to_club]["transfer_fee"] += player[1][-index]["fee"]
                        G[from_club][to_club]["num_transfers"] += 1
                    else:
                        G.add_edge(from_club, to_club, transfer_fee=player[1][-index]["fee"], num_transfers=1)

In [112]:
df = pd.DataFrame(transfer, columns= ['from_club_id', 'to_club_id','player_nationality','value_increase','player_stay_in_years','club_market_value_from','club_market_value_to','transfer_year','position','birth_date_year'])
df

,from_club_id,to_club_id,player_nationality,value_increase,player_stay_in_years,club_market_value_from,club_market_value_to,transfer_year,position,birth_date_year
0,1184,631,Belgium,8950000,2,63300000,631900000,2011,Goalkeeper,1992
1,631,13,Belgium,-7750000,0,631900000,509500000,2011,Goalkeeper,1992
2,13,631,Belgium,23800000,3,509500000,631900000,2014,Goalkeeper,1992
3,1084,281,Argentina,7100000,3,67580000,629500000,2014,Goalkeeper,1981
4,281,631,Argentina,-8000000,3,629500000,631900000,2017,Goalkeeper,1981
5,1075,1085,Portugal,0,0,53350000,15400000,2007,Goalkeeper,1982
6,1085,1075,Portugal,1500000,1,15400000,53350000,2008,Goalkeeper,1982
7,1075,252,Portugal,3000000,2,53350000,87900000,2010,Goalkeeper,1982
8,252,294,Portugal,2000000,1,87900000,165850000,2011,Goalkeeper,1982
9,294,252,Portugal,-2500000,1,165850000,87900000,2012,Goalkeeper,1982


In [113]:
nx.write_adjlist(G, "transfers_graph.adjlist")
df.to_csv("transfers.csv")